In [1]:
import numpy as np
from scipy import optimize
import sympy as sm
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
import my_model
%load_ext autoreload
%autoreload 2

In [2]:
#Updating settings
plt.rcParams.update({"axes.grid": True, "grid.color": "black", "grid.alpha": "0.25", "grid.linestyle": "--"})
plt.rcParams.update({'font.size': 14})


# Model Description

The objective of this model analysis project is to demonstrate proficiency in applying numerical model analysis methods, structuring a code project, documenting code, and presenting results in text form and figures. The project involves analyzing an economic model, solving it using optimization routines or equation solvers, simulating the model, visualizing the solution across different parameterizations, and analyzing one or more extensions of the baseline model.

In this project, we have chosen to work with the AD-AS (Aggregate Demand-Aggregate Supply) model, which is a fundamental model in macroeconomics. The AD-AS model provides insights into the relationship between output (real GDP) and inflation in an economy.

To solve the AD-AS model, we have utilized numerical methods implemented in Python. The code is structured in a self-contained notebook (.ipynb) format, accompanied by fully documented Python files (.py). 

The main steps of the analysis are as follows:

Algorithm Description: We describe the algorithm used to solve the economic model, explaining the equations and relationships involved.
Solving the Model: We employ numerical optimization and equation solving techniques to find the equilibrium solutions of the AD-AS model. This involves determining the steady-state equation, converting it into a function, and solving for the equilibrium output level.
Visualization of Solutions: We plot the AD and AS curves, showcasing the relationship between output and inflation. The interactive plot allows us to adjust the shocks and observe the impact on the model.
Extension of the Baseline Model: To further explore the model, we introduce an extension by incorporating a monetary policy rule known as the Taylor Rule. This allows us to analyze the effects of different monetary policy settings on equilibrium output and inflation.
The limitations of the project include the minimal use of sympy, as the focus is primarily on numerical solutions. The emphasis is on quality rather than quantity, ensuring a well-tested model with effective visualization and simulation.

In conclusion, this project demonstrates the application of numerical model analysis methods to the AD-AS model, showcasing the impact of different parameterizations and monetary policy settings on equilibrium output and inflation. Through proper code structuring, documentation, and visualization, the project presents a comprehensive analysis of the model, fulfilling the objectives outlined in the assignment description.

# Numerical Solution of the AD-AS model

In [3]:
# Define symbols
y, pi, g, b, alpha1, alpha2, h, s, y_bar, pi_star = sm.symbols('y pi g b alpha1 alpha2 h s y_bar pi_star')
z = alpha1 / (1 + alpha2 * b) * (g - y_bar)
alpha = alpha2 * h / (1 + alpha2 * b)
AD = y - y_bar - alpha * (pi - pi_star) - z

gamma = sm.symbols('gamma')
AS = pi - pi_star + gamma * (y - y_bar) + s


In [4]:
# Solve steady state equation
steady_state_eq = sm.solve(sm.Eq(AD, AS), y)[0]


In [5]:
# Convert equation to a function
ss_func = sm.lambdify((pi, g, b, alpha1, alpha2, h, s, y_bar, pi_star, gamma), steady_state_eq)

In [6]:
# Define AD curve function
def ad(y, pi, pi_star, alpha, alpha_1, alpha_2, b, g, g_bar, h):
    z = alpha_1 / (1 + alpha_2 * b) * (g - g_bar)
    return y - y_bar - z + alpha * (pi - pi_star)

# Define AS curve function
def as_curve(y, pi, pi_1, gamma, s):
    return pi - pi_1 - gamma * (y - y_bar) - s

In [7]:
# Set parameter values
y_bar = 100  # Steady-state output level
pi_star = 2  # Steady-state inflation rate
alpha_1 = 1  # Coefficient determining the effect of government spending on output
alpha_2 = 1  # Coefficient determining the sensitivity of output to inflation
b = 1  # Parameter representing a fixed input-output ratio
g = 1  # Government spending
g_bar = 1  # Steady-state government spending
h = 1  # Parameter determining the effect of output on inflation in the AS curve

alpha = alpha_2 * h / (1 + alpha_2 * b)
gamma = 0.5

In [8]:
# Plot AD and AS curves
def plot_ad_as(s):
    y_range = np.linspace(y_bar - 10, y_bar + 10, 100)
    ad_curve_values = ad(y_range, pi_star, pi_star, alpha, alpha_1, alpha_2, b, g, g_bar, h)
    as_curve_0 = as_curve(y_range, pi_star, pi_star, gamma, 0)
    as_curve_1 = as_curve(y_range, pi_star, pi_star, gamma, s)
    
    plt.plot(y_range, ad_curve_values, label='AS')
    plt.plot(y_range, as_curve_0, label='AD (s=0)', color='grey')
    plt.plot(y_range, as_curve_1, label='AD (s={:.2f})'.format(s), color='red')

    plt.xlabel('Output')
    plt.ylabel('Inflation')
    plt.legend()

# Create interactive plot
interact(plot_ad_as, s=FloatSlider(min=-5, max=5, step=0.1, value=0));


interactive(children=(FloatSlider(value=0.0, description='s', max=5.0, min=-5.0), Output()), _dom_classes=('wi…

In [9]:
# Set initial parameter values
pi = 0.02
g = 0.03
b = 0.5
alpha1 = 1.5
alpha2 = 1
h = 0.5
s = 0
y_bar = 1
pi_star = 0.02
gamma = 0.5

# Define social loss function
def social_loss(par, *args):
    pi, g, b, alpha1, alpha2, h, s, y_bar, pi_star, gamma = args
    y = ss_func(pi, g, b, alpha1, alpha2, h, s, y_bar, pi_star, gamma)
    return (y_bar - y) ** 2 + (pi_star - pi) ** 2

# Optimize social loss function
optimal_par = optimize.minimize(social_loss, [pi, g, b, alpha1, alpha2, h, s, y_bar, pi_star, gamma],
                                args=(pi, g, b, alpha1, alpha2, h, s, y_bar, pi_star, gamma), method='Nelder-Mead')

# Extract the optimal parameter values
optimal_parameters = optimal_par.x

# Print optimal parameter values
print("The optimal parameter values that minimize the social loss are:")
for parameter, value in zip(["pi", "g", "b", "alpha1", "alpha2", "h", "s", "y_bar", "pi_star", "gamma"], optimal_parameters):
    print(f"{parameter}: {value}")


The optimal parameter values that minimize the social loss are:
pi: 0.02
g: 0.03
b: 0.5
alpha1: 1.5
alpha2: 1.0
h: 0.5
s: 0.0
y_bar: 1.0
pi_star: 0.02
gamma: 0.5


# Further Analysis: Extension of the model 

### Monetary Policy Rule

We introduce a monetary policy rule that governs the behavior of the central bank. This extension allows us to analyze the effects of monetary policy on the economy.

In [10]:
#We define a monetary policy rule that determines the nominal interest rate based on the inflation rate; The Taylor Rule
def monetary_policy_rule(inflation, inflation_target, output_deviation, interest_rate_target):
    return interest_rate_target + 0.5 * (inflation - inflation_target) + 0.5 * output_deviation


In [11]:
#We update the AD-curve to include the monetary rule
def ad(y, pi, pi_star, alpha, alpha_1, alpha_2, b, g, g_bar, h, inflation_target, interest_rate_target):
    z = alpha_1 / (1 + alpha_2 * b) * (g - g_bar)
    output_deviation = y - y_bar
    nominal_interest_rate = monetary_policy_rule(pi, inflation_target, output_deviation, interest_rate_target)
    return y - y_bar - z + alpha * (pi - pi_star) + nominal_interest_rate

In [12]:
#We update the plot to include the inflation target and interest rate target as arguments:
def plot_ad_as(s, inflation_target, interest_rate_target):
    y_range = np.linspace(y_bar - 10, y_bar + 10, 100)
    ad_curve = ad(y_range, pi_star, pi_star, alpha, alpha_1, alpha_2, b, g, g_bar, h, inflation_target, interest_rate_target)
    as_curve_0 = as_curve(y_range, pi_star, pi_star, gamma, 0)
    as_curve_1 = as_curve(y_range, pi_star, pi_star, gamma, s)
    
    plt.plot(y_range, ad_curve, label='AS')
    plt.plot(y_range, as_curve_0, label='AD (s=0)', color='grey')
    plt.plot(y_range, as_curve_1, label='AD (s={:.2f})'.format(s), color='red')

    plt.xlabel('Output')
    plt.ylabel('Inflation')
    plt.legend()

#The interact function allows us to create an interactive plot that allows us to adjust the AD shocks, inflation target and interest rate target
interact(plot_ad_as, s=FloatSlider(min=-5, max=5, step=0.1, value=0), inflation_target=FloatSlider(min=0.01, max=0.20, step=0.01, value=0.02), interest_rate_target=FloatSlider(min=0.01, max=0.20, step=0.01, value=0.03));


interactive(children=(FloatSlider(value=0.0, description='s', max=5.0, min=-5.0), FloatSlider(value=0.02, desc…

**Interpretation of extension with a monetary policy rule**

By adjusting the inflation target, we can examine the impact of different inflation goals on the equilibrium output and inflation. Higher inflation targets are expected to lead to higher equilibrium inflation rates and potentially higher output levels, as the central bank seeks to stimulate economic activity. Conversely, lower inflation targets may result in lower equilibrium inflation rates and output levels, as the central bank aims for price stability.

Similarly, adjusting the interest rate target allows us to analyze the influence of different monetary policy stances on the AD-AS model. Higher interest rate targets indicate a tighter monetary policy, which can dampen inflationary pressures and potentially lower output. Lower interest rate targets, on the other hand, signal an accommodative monetary policy that encourages borrowing and spending, potentially leading to higher inflation and output.

Equilibrium Output:
Higher inflation and lower interest rates lead to higher equilibrium output, while lower inflation and higher interest rates result in lower equilibrium output.

Inflation Dynamics:
Higher inflation targets and accommodative interest rate policies contribute to higher inflation, while lower targets and tighter policies can help maintain price stability with lower inflation rates.

Trade-offs between inflation and output: Expansionary monetary policy measures aimed at boosting output and economic growth may come at the cost of higher inflation, while contractionary policies aimed at curbing inflation may result in lower output levels.

# Conclusion

In this code, we have implemented the AD-AS model and extended it to include a monetary policy rule, known as the Taylor Rule. The model is solved using numerical methods.

We start by defining the AD and AS curves based on given equations. The steady-state equation is solved to determine the equilibrium output level. The AD and AS curves are then plotted, showing the relationship between output and inflation.

Next, we introduce the monetary policy rule, which determines the nominal interest rate based on the inflation rate. The AD curve is updated to incorporate the monetary rule, considering the inflation target and interest rate target as additional arguments. We plot the updated AD and AS curves, allowing us to analyze the effects of different monetary policy settings on the model.

To further analyze the model, we define a social loss function and optimize it to find the parameter values that minimize the social loss. This optimization process is carried out using the Nelder-Mead method.

The conclusion from this model analysis is that changes in inflation targets and interest rate targets have significant effects on equilibrium output and inflation. Higher inflation targets and accommodative interest rate policies tend to increase output and inflation, while lower targets and tighter policies have the opposite effect. The model demonstrates the trade-offs between inflation and output and highlights the role of monetary policy in influencing macroeconomic outcomes.

Overall, this model analysis provides insights into the impact of monetary policy rule on the dynamics of the AD-AS model and highlights the importance of understanding the effects of different policy settings on macroeconomic variables.



